**Using BERTweet-base-finetuned-SARC-combined-DS to label S3D**

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("surrey-nlp/bertweet-base-finetuned-SARC-combined-DS")

model = AutoModelForSequenceClassification.from_pretrained("surrey-nlp/bertweet-base-finetuned-SARC-combined-DS")

Downloading:   0%|          | 0.00/338 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/819 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

In [2]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 100)
df = pd.read_csv('/kaggle/input/s3dpreprocessed/S3D-preprocessed.csv', lineterminator = '\n')
df = df['Text']
df = df.dropna(axis=0, how="any", inplace=False)

In [3]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, truncation=True, device= 0, binary_output=True)
data = list(df)
out = pipe(data)

/opt/conda/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning


In [4]:
def get_preds(out):

    labels = []
    for i in range(len(out)):
        if out[i][0]['score'] > out[i][1]['score']:
            labels.append(0)
        else:
            labels.append(1)
            
    return labels

In [5]:
labels = get_preds(out)
df2 = pd.DataFrame(labels, columns = ['Label'])
df3 = pd.concat([df, df2], axis = 1)

In [6]:
df3[0:100]

,Text,Label
0,@user new jersey not the big apple smh admin,1.0
1,the uk @user some 28bn of total wealth most of it tied up in real estate eg buckingham broad exe...,0.0
2,@user the feeling of a country being forced to mourn,1.0
3,@user @user @user i spend the entire summer by changing my life style like exercising right eati...,0.0
4,ill make a karma cometrueyeaahhh idgaf but at least u felf it cuz all i know the world is a veng...,1.0
5,we are thrilled to announce the first family fare event of the season donna washington is an in...,0.0
6,europe holds emergency talks as electricity prices soar the wall street journal,1.0
7,hey folks its mommy government here the weather is a bit warm out so were not allowing you to us...,1.0
8,man looks like charles is going all the way down this line,0.0
9,@user sorry to hear that we understand your concern related to the order please be assured we ar...,0.0
